## 성능개선
- 베이스 라인 모델 성능을 개선해보겠다. LightGBM
- 하이퍼 파라미터 최적화, 피처 엔지니어링이 있는데, 이전에 하이퍼 파라미터 최적화 많이 해봤으므로, 피처 엔지니어링에 집중하겠다.
- 다양한 피처를 만들고 피처 엔지니어링 해보자.

### 피처엔지니어링
1. 베이스 라인과 똑같이 피처명 한글화와 데이터 다운캐스팅
2. 개별 데이터, sales_train, shops, items, item_categories를 활용해 전처리, 파생 피처 생성, 인코딩을 수행함.
3. 베이스라인과 동일하게 데이터 조합을 만듦. + 파생 피처 추가함.
4. 테스트 데이터를 합친 후, 2번에서 피처 엔지니어링한 다른 데이터를 병합함.
5. 시차 피처를 만듦. 시차 피처를 만들면 먼저 '기준 피처별 월간 평균 판매량 피처'를 구해야함. 이 피처를 기준으로 시차 피처를 만들것임.
6. 그외 추가적인 피처 엔지니어링을 적용함.

### 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고 메시지 생략

# 데이터 경로
data_path = './data/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 1-1. 피처 엔지니어링 : 피처명 한글화와 데이터 다운캐스팅
- 베이스 라인에서 한 것들.

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

item_categories = item_categories.rename(columns= {'item_category_name': '상품분류명',
                                                   'item_category_id': '상품분류ID'})

test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [3]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

all_df = [sales_train, shops, items, item_categories, test]

for df in all_df:
    df = downcast(df)

54.2% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


### 1-2. 피처 엔지니어링 2 : 개별 데이터 피처 엔지니어링
- sales_train
- shops
- items
- item_categories
데이터를 각각 피처 엔지니어링함.

#### 1-2-1. sales_train 이상치 제거 및 전처리
- 판매가와 판매량 피처의 이상치를 제거할 것임. -> 이상치 있으면 성능 나빠지니까

#### 그럼 이상치는?
1. 판매가, 판매량이 음수라면 환불이거나 오류임. 따라서 음수는 이상치.
2. 데이터 분석했듯이 판매가가 50,000 이상 / 판매량이 1,000 이상인 데이터도 이상치로 간주.

In [4]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1


In [5]:
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가'] > 0]

# 판매가가 50,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가'] < 50000]

# 판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매량'] > 0]

# 판매량이 1,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매량'] < 1000]

#### 그럼 전처리는?
- 상점명을 기준으로. 
- 러시아어라서 잘 모르겠지만, 코드를 참고하면, 다음 상점명 네 쌍이 같은 의미가 된다.
- 이들의 상점명이 유사하기 때문인데, 한번 비교해본다.

In [6]:
print(shops['상점명'][0], '||', shops['상점명'][57])

print(shops['상점명'][1], '||', shops['상점명'][58])

print(shops['상점명'][10], '||', shops['상점명'][11])

print(shops['상점명'][39], '||', shops['상점명'][40])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


- 러시아어를 잘 모르겠지만 아무튼 두 상점 명이 비슷함을 알 수 있다.
- 비슷한 두 상점은 다음과 같이 수정한다.

1. 상점ID : 0 -> 57 / 1 -> 58 / 10 -> 11 / 39 -> 40
2. 상점ID만 수정하는 이유는 어차피 상점명은 일대일 대응이라 제거할 것이미 때문.

In [7]:
# sales_train 데이터에서 상점ID 수정
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# test 데이터에서 상점ID 수정
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

##### 1-2-2. shop 파생 피처 생성 및 인코딩
- shop에서 새로운 피처 만들고 인코딩하기
- 다른 캐글러가 상점명의 첫단어가 도시라는 사실을 알아냄. -> 상점명을 이용해 도시 피처를 만들 수 있음.
- 공백 기준으로 상점 명 피처를 나누고, 첫번째 단어를 가져오면 됨.

In [8]:
shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
# 모델의 성능을 더 향상시키는 방법 - 실전문제
# 테스트 데이터에 있는 상점ID만 추출
unique_test_shop_id = test['상점ID'].unique()
sales_train = sales_train[sales_train['상점ID'].isin(unique_test_shop_id)]

In [10]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [11]:
# 파생피처 : 도시
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [12]:
# 맨 처음 도시명 앞에 느낌표는 잘못 기재된거라 바꿔주기
shops.loc[shops['도시'] =='!Якутск', '도시'] = 'Якутск'

- 도시 피처는 범주형 피처인데, 문자를 숫자로 바꿔서 인식하기 쉽게 만들어주자. (머신러닝 모델은 문자를 인식하지 못한다.)
- 인코딩을 이용하기!
    - 레이블 인코딩
        단점 : 서로 가까운 숫자를 비슷한 데이터로 판단함. 하지만, 트리 기반 모델을 사용할 땐 문제 없음. 트리 기반 모델 특성상 분기를 반복하면서 피처 정보를 반영하므로, 레이블 인코딩 단점이 어느정도 무마됨.

In [13]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [14]:
shops.head()

,상점명,상점ID,도시
0,"!Якутск Орджоникидзе, 56 фран",0,29
1,"!Якутск ТЦ ""Центральный"" фран",1,29
2,"Адыгея ТЦ ""Мега""",2,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,1
4,"Волжский ТЦ ""Волга Молл""",4,2


- 이전에 말했듯이 상점 명 피처를 제거함.

In [15]:
# 상점명 피처 제거
shops = shops.drop('상점명', axis=1)

shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


##### 1-2-3. items 파생 피처 생성
- 첫 판매월 피처를 구해보자.
- 상품명과 상품ID는 일대일 대응이므로 제거해준다.

In [16]:
items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [17]:
# 상품명 피처 제거
items = items.drop(['상품명'], axis=1)

- 처음 팔린 월을 피처로 만들겠다. -> items 만으로는 구할 수 없음! sales_train 데이터 이용할 것임.
- sales_train을 상품ID 기준으로 그룹화하고, 그룹에서 월ID의 최솟값을 구하면 됨.

In [18]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID': 'min'})['월ID']

items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,NaN
1,1,76,15.0
2,2,40,NaN
3,3,40,NaN
4,4,40,NaN


문제는 이 테이블에 결측값이 존재한다. 결측값을 확인해보자.

In [19]:
# 결측값 확인
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
0,0,40,NaN
2,2,40,NaN
3,3,40,NaN
4,4,40,NaN
5,5,40,NaN
...,...,...,...
21975,21975,61,NaN
22022,22022,40,NaN
22028,22028,40,NaN
22035,22035,40,NaN


368개의 데이터에 결측값이 있다.
당연히 해당 상품이 전혀 판매된 적이 없으므로 옳다.

- 한 번도 팔리지 않은 상품의 결측값을 34로 대체한다. (train은 0~33 까지)
- 물론, 2015.11(34)에도 안팔릴 수 있는데, 문제되지 않음. 어차피 테스트 데이터에 없는 상품이면 고려대상이 아니기 때문.
- 사실 어떤 값이든 상관없음. 

In [20]:
# 첫 판매월 피처의 결측값을 34로 대체
items['첫 판매월'] = items['첫 판매월'].fillna(34)

##### 1-2-4. item_categories 파생 피처를 생성하고 인코딩
- '대분류'라는 파생 피처를 만들고, 인코딩해보자.
- 상품분류명을 보면, 첫 단어가 범주 대분류라는 점을 알 수 있다. (다른 캐글러의 도움!)

In [21]:
item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


큰 범주로 묶는데, 문제는 전체 데이터 개수는 고정되어 있으므로 범주형 데이터가 지나치게 세밀하면 자연스럽게 각 범주에 해당하는 데이터의 개수가 부족해서 제대로 훈련되지 않을 수 있다. 하지만 큰 범주로 묶으면 각 범주별 훈련 데이터 수가 많아져서 예측 정확도가 높아짐. 

따라서 큰 범주로 묶으면 범주가 지나치게 세밀할 때보다 성능 향상에 유리함.

In [22]:
# 상품분류명의 첫 단어를 대분류로 추출
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])  

In [23]:
# 대분류의 고윳값 별로 데이터의 개수
item_categories['대분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: 대분류, dtype: int64

- 참고로, 맨 위부터, 게임, 책, 선물을 의미한다.
- 고윳값이 5개 미만인 대분류는 모두 etc로 묶는다. 세밀하면 성능이 나쁘니까.

In [24]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# 대분류의 고윳값 개수가 5개 미만이면 'etc'로 바꾸기
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


- 이해하기 어려운 러시아어를 인코딩 한다. 
=> 역시 레이블 인코딩으로 한다!

In [25]:
# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 대분류 피처 레이블 인코딩
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

# 상품분류명 피처 제거, 사유 : ID와 일대일 대응
item_categories = item_categories.drop('상품분류명', axis=1)


item_categories['대분류'].head()

0    0
1    1
2    1
3    1
4    1
Name: 대분류, dtype: int64

### 각 데이터에 적용한 피처 엔지니어링 정리
#### sales_train
- 이상치 제거
- 상품ID 네 쌍 수정

#### shops
- 상점명 피처를 활용해 도시 피처 새로 추가
- 범주형 데이터인 도시 피처 인코딩
- 상점명 피처 제거

#### items
- 첫 판매월 피처 추가
- 결측값을 34로 대체함.

#### item_categories
- 대분류 피처 추가
- 고윳값 개수 5개 미만인 대분류 값을 'etc'로 변경
- 대분류 피처 인코딩

### 1-3 피처 엔지니어링 3 : 데이터 조합 및 피처 생성
- 베이스라인과 동일하게 데이터 조합을 생성함.
- 월간 판매량, 평균판매가, 판매건수 피처를 추가할 것임.

In [26]:
# 베이스라인과 동일한 방식으로 월ID, 상점ID, 상품ID 조합을 생성하기.
from itertools import product

train = []

# 월ID, 상점ID, 상품ID 조합 생성
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [27]:
# 월간 판매량, 평균 판매가, 판매건수 파생피처 추가하기
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '월간 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


- 결측값이 있는데, 결측값이 있다는 건 판매량과 판매가가 0이라는 뜻이다. -> 0으로 수정
- group은 임시로 만든 변수이므로 가비지 컬렉션 해줌.

In [28]:
import gc

# group 변수 가비지 컬렉션
del group
gc.collect();

- 기준 피처별 상품 판매건수 파생 피처를 추가해볼 것임.
    - 상품 월간 판매량과 다른 개념임. 상품 판매량이 이틀전 3개, 어제 0개, 오늘 2개라면 월간 판매량은 5개
    - 판매건수는 2건임(이틀전, 오늘) 즉 판매 못하면(0이면) 0건임. 집계함수 count로 기준 피처별 상품 판매건수 구할 수 있다.

In [29]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

# 가비지 컬렉션
del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


판매건수 결측값도 0으로 설정함.

### 1-4. 피처 엔지니어링 4 : 데이터 합치기
- 테스트 데이터를 붙이고, 피처 엔지니어링한 sales_train, shops, items, item_categories 데이터를 병합하겠다.

#### 1-4-1. 테스트 데이터 이어붙이기
- train에 테스트 데이터를 이어붙임. 베이스라인과 동일한 코드

In [30]:
# 동일한 베이스 라인 코드

# 테스트 데이터 월ID를 34로 설정
test['월ID'] = 34

# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)

In [31]:
# 이전에 말한 모든 결측값 0으로 설정하기

# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


#### 1-4-2. 모든 데이터 병합
- all_data에 shops, items, item_categories 데이터를 병합하기.
- 이어서 데이터 다운 캐스팅도 진행함.

In [32]:
# 동일한 베이스 라인 코드

# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

60.9% 압축됨


In [33]:
# 모든 데이터 압축했으니, 필요없는 데이터 가비지 컬렉션

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

### 1-5. 피처 엔지니어링5 : 시차 피처 생성
- 시차피처(time lag feature) : 과거 시점에 대한 피처, 성능 향상에 도움이 되는 경우가 많아 시계열 문제에서 자주 만드는 파생피처.

#### 어떻게 만드는데?
1. 징검다리 피처 생성 (기준 피차별 월간 평균 판매량) -> 기준으로 삼을 피처 정하기 - 타깃값과 관련된 '월간 평균 판매량' 피처 필요.
2. 시차 피처 생성

##### 1. 기준 피처별 월간 평균 판매량 파생 피처 생성
- 월간 평균 판매량을 구할 때 기준 피처는 다를 수 있음. 
    1. 상점별 월간 평균 판매량
    2. 상품별 월간 평균 판매량
    3. 각 상점의 상품별 월간 평균 판매량
    
    
- 만드는데 필요한 파라미터
    1. df : 작업할 전체 데이터(DataFrame)
    2. mean_features : 새로 만든 월간 평균 판매량 파생 피처명을 저장하는 리스트
    3. idx_features : 기준 피처


- 기준 피처는 반드시 '월ID' 여야한다. 당연히 '월간' 평균 판매량 파생 피처를 만들거니까.
- 기준 피처가 너무 많으면 과도하게 세분화 되므로, 2개 또는 3개로 정함.

In [34]:
def add_mean_features(df, mean_features, idx_features):
    
    # 기준 피처 확인 
    # 기준 피처의 첫 번째 요소가 '월ID'가 맞는지, 기준 피처 개수가 2개 혹은 3개 인지 확인한다. 아닐 경우 오류 발생.
    assert (idx_features[0] == '월ID') and \
           len(idx_features) in [2, 3]
    
    # 파생 피처명 설정 
    # 기준 피처 개수가 2개일 때와 3개일때로 나누어 설정함.
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기 
    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    
    # 이름도 이전에 정한 이름으로 바꿔준다.
    group = group.rename(columns={'월간 판매량': feature_name})
    
    # df와 group 병합 
    # 원본 데이터 기준으로 병합하므로 병햡 조건이 left임.
    # 파생피처가 원본 데이터에 추가됨.
    df = df.merge(group, on=idx_features, how='left')
    
    # 데이터 다운캐스팅 
    # verbose = False로 설정하여 몇 퍼센트 압축했는지 문구를 출력해주지 않음.
    df = downcast(df, verbose=False)
    
    # 새로 만든 feature_name 피처명을 mean_features 리스트에 추가 
    # 추후 시차 피처 만드는데 사용할 것임.
    mean_features.append(feature_name)
    
    # 가비지 컬렉션
    del group
    gc.collect()
    
    return df, mean_features

In [35]:
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
# 추후, 이 리스트에 저장된 파생 피처를 활용하여 추가적인 피처 엔지니어링(시차 피처 생성)을 적용하는데 사용할 것임.
item_mean_features = []

# case 1. ['월ID', '상품ID']로 그룹화 하여 만들기
# ['월ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])


# case 2. ['월ID', '상품ID', '도시']로 그룹화 하여 만들기
# ['월ID', '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [36]:
# '상품ID'가 포함된 파생피처 모음
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [37]:
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 피처명을 담을 리스트
shop_mean_features = []

# case 1. ['월ID', '상점ID', '상품분류ID']로 그룹화 하여 만들기
# ['월ID', '상점ID', '상품분류ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [38]:
# '상점ID'가 포함된 파생피처 모음
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

지금까지 만든 파생피처로 시차 파생피처를 생성하겠다.

#### 시차 피처 생성 원리 및 함수 구현
1. {상품ID}별 평균 판매량
2. {상품ID + 도시}별 평균 판매량
3. {상점ID + 상품범주ID}별 평균 판매량

- 세 징검다리 피처를 사용해 시차 피처를 구해보자.
- 시차피처는 사계열 문제에서 자주 사용하는 파생피처임. 현시점 데이터에 과거 시점 데이터를 추가하는 개념.
- 과거 시점 데이터는 향후 판매량 예측에 유용하기 때문임.
- 시차 피처는 한 달전, 두 달전,세 달전 등 원하는 시점까지 생성 가능하나, 너무 과거면 예측력이 떨어지므로 세 달전까지만 만들겠음.

#### 시차 피처 생성 방법 예시 : 한달 전
1. 기준 피처와 '시차값을 구하려는 피처'를 정한다. 여기서는 기준 피처를 '월ID', '상점ID', '상품ID'로 정하고, 시차값을 구하려는 피처를 '월간 판매량'으로 정했음.
2. 원본 데이터인 df에서 기준 피처와 월간 판매량 피처만 추출해 복사본을 만듦. 이를 df_temp라 함.
3. 새로 만들 시차 피차명을 정함. '월간 판매량_시차1'로 정함.
4. df_temp에서 '월간 판매량' 피처명을 '월간 판매량_시차1'로 바꿈.
5. df_temp의 '월ID' 피처에 1을 더함. (한달 시차를 생성하는 작업) *여기가 중요!
6. 기준 피처를 토대로 df와 df_temp를 병합함.


이 과정을 통해 df에 한 달 전 시차 피처가 생성된다.

In [39]:
# 파라미터 
# - df : 원본 데이터
# - lag_features_to_clip : 값의 범위를 0~20 사이로 제한할 피처를 담을 리스트
# - idx_features : 기준 피처
# - lag_feature : 시차를 만들 피처
# - nlags : 시차
#   - 1 : 한달 전 2 : 한 달 ~ 두 달전 3 : 한 달 ~ 세 달전
# - clip : 새로 만든 시차 피처를 lag_features_to_clip 리스트에 저장할 지 여부 (True or False)
#   - lag_features_to_clip 리스트에 들어있는 피처 값은 나중에 0~20 사이로 제한됨.

def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사 
    # 원본 데이터인 df에서 원하는 피처만 추출해 복사본을 만듦.
    # idx_features는 리스트 피처, lag_feature는 문자열 피처이므로, []를 추가하여 리스트에 다 넣음. (원하는 피처만 추출)
    # copy()로 복사본 만들어서 temp에 넣음.
    df_temp = df[idx_features + [lag_feature]].copy() 

    # 시차 피처 생성 
    # nlags 인수 만큼 for문을 돌며 시차 피처를 생성한다.
    for i in range(1, nlags+1):
        
        # 시차 피처명 지정
        lag_feature_name = lag_feature +'_시차' + str(i)
        
        # df_temp 열 이름 설정 
        # lag_feature 명을 아까 지저한 시차 피처명으로 지정함.
        df_temp.columns = idx_features + [lag_feature_name]
        
        # df_temp의 date_block_num 피처에 1 더하기 
        # 핵심 : 피처값은 그대로인데(값은 i), 병합하는 기준이 되는 월ID는 i+1이므로, 값만 정확히 1이전인 것이다. 
        df_temp['월ID'] += 1
        
        # idx_feature를 기준으로 df와 df_temp 병합하기 
        # df_temp에 중복된 행이 있을 수 있으니, drop_duplicates()로 중복된 행은 제거한 뒤 병합함.
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        
        # 결측값 0으로 대체 
        # 매달 데이터가 반드시 있는게 아니므로, 한 달전 피처가 없을 수 있음. 이때 생긴 결측값은 0으로 대체.
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        
        # 0 ~ 20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가 
        # 방근 만든 시차 피처명을 clip인수가 True일 경우,리스트에 추가해줌. (clip하려고)
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # 데이터 다운캐스팅
    df = downcast(df, False)
    
    # 가비지 컬렉션
    del df_temp
    gc.collect()
    
    # 시차피처가 추가된 데이터 df와 clipping할 시차 피처명이 반한됨.
    return df, lag_features_to_clip

#### 1-5-1. 시차 피처 생성 1 : 월간 판매량
- 기준 피처 : '월ID', '상점ID', '상품ID'
- 세달치 시차 피처 생성
- 월간 판매량은 타겟값 이므로, 0~20 사이로 제한해야하므로, 추가된 시차피처는 clipping하기 위해 True로 설정.

In [40]:
lag_features_to_clip = [] # 0 ~ 20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 월간 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량', 
                                                  nlags=3,
                                                  clip=True) # 값을 0 ~ 20 사이로 제한

In [41]:
# 시차 피처가 잘 만들어졌는지 보자.
# 피처가 많아서 행과 영을 바꿔 출력함.
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


- 월간_판매량_시차1, 월간_판매량_시차2, 월간_판매량_시차3 피처가 잘 만들어짐.
- 3개의 시차 피처를 만듦.

In [42]:
# clipping할 피처 목록들.
lag_features_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

#### 1-5-2. 시차 피처 생성 2 : 판매건수, 평균 판매가
- 기준 피처 : '월ID', '상점ID', '상품ID'
- 세달치 시차 피처 생성
- 판매건수, 평균 판매가는 타깃값이 아니므로 0~20 사이로 제한할 필요 없음. False

In [43]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수', 
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가', 
                                                  nlags=3)

In [44]:
# 지금까지 시차 피처 결과 확인하기
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


#### 1-5-3. 시차 피처 생성 3 : 평균 판매량
- 상품별 평균 판매량, 상점별 평균 판매량 등등으로 파생 피처를 만든 적 있음. : item_mean_features
- 두 피처 값에 대해서도 시차 피처를 만든다.

In [46]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [47]:
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3,
                                                      clip=True)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

시차피처 만든후, item_mean_features를 제거함. 이들은 시차 피처 만드는데만 필요했으므로.

In [48]:
# 시차 피처 결과 확인하기 : 둘 다 만들어 졌음을 확인할 수 있다.
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.0,0.0,0.000000,0.000000
상점ID,59.000000,59.0,59.0,59.000000,59.000000
상품ID,22154.000000,2552.0,2554.0,2555.000000,2564.000000
월간 판매량,1.000000,0.0,0.0,0.000000,0.000000
평균 판매가,999.000000,0.0,0.0,0.000000,0.000000
판매건수,1.000000,0.0,0.0,0.000000,0.000000
도시,30.000000,30.0,30.0,30.000000,30.000000
상품분류ID,37.000000,58.0,58.0,56.000000,59.000000
첫 판매월,0.000000,0.0,0.0,0.000000,0.000000
대분류,5.000000,7.0,7.0,7.000000,7.000000


In [49]:
# 파생 피처가 하나만 저장됨.
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

In [50]:
# ['월ID', '상점ID', '상품분류ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=['월ID', '상점ID', '상품분류ID'],
                                                      # 분류 기준을 idx_features로 해도 됨. 어차피 피처는 많으니.
                                                      lag_feature=shop_mean_feature, 
                                                      nlags=3,
                                                      clip=True)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

In [51]:
# 시차 피처 결과 확인하기 : 완성!
all_data.head().T

,0,1,2,3,4
월ID,0.0,0.0,0.0,0.0,0.0
상점ID,59.0,59.0,59.0,59.0,59.0
상품ID,22154.0,2552.0,2554.0,2555.0,2564.0
월간 판매량,1.0,0.0,0.0,0.0,0.0
평균 판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1.0,0.0,0.0,0.0,0.0
도시,30.0,30.0,30.0,30.0,30.0
상품분류ID,37.0,58.0,58.0,56.0,59.0
첫 판매월,0.0,0.0,0.0,0.0,0.0
대분류,5.0,7.0,7.0,7.0,7.0


시차피처 만든후, shop_mean_features를 제거함. 이들은 시차 피처 만드는데만 필요했으므로.

#### 1-5-3. 피처 생성 마무리 : 결측값 처리
- 시차 피처 3개월치를 만듦. -> 자연스럽게 0, 1, 2에는 결측값이 생길 수 밖에.
- 3개월 미만인 데이터를 제거하여 결측값을 없앰.

In [52]:
# 월ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

### 1-6. 피처 엔지니어링 6 : 기타 피처 엔지니어링
- 간단한 피처 몇개를 추가하고, 필요없는 피처 제거할 것임.

#### 1-6-1. 기타 피처 추가
1. 월간 판매량 시차 피처들의 평균 : 월간 판매량 시차 평균.
2. 시차 변화량.
3. 신사 여부 : 첫 판매월이 현재 월과 같으면 신상품임.
4. 첫 판매 후 (경과) 기간 : 현재 월 - 첫 판매월.
5. 월 : 월ID % 12

In [53]:
# case 1. 월간 판매량 시차 평균
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2', 
                                          '월간 판매량_시차3']].mean(axis=1)

# '월간 판매량'과 관련 있으므로, clipping이 필요함. 
# 0 ~ 20 사이로 값 제한
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] \
= all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [54]:
# case 2. 월간 판매량 시차 평균
# 1 -> 2
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']

# 값을 0으로 나누는 상황을 대처하는 방어코드 양수를 0으로 나누면 무한대, 음수를 0으로 나누면 무한소가 된다. 0/0은 0으로 맞춰줌.
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

# 2 -> 3
all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']

# 값을 0으로 나누는 상황을 대처하는 방어코드 양수를 0으로 나누면 무한대, 음수를 0으로 나누면 무한소가 된다. 0/0은 0으로 맞춰줌.
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

In [55]:
# case 3. 신상 여부
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

In [56]:
# case 4. 첫 판매 후 기간
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [57]:
# case 5. 월 : 12월은 0월임.
all_data['월'] = all_data['월ID'] % 12

In [58]:
# 기타 피처 추가한 결과 확인
all_data.head().T

,827196,827197,827198,827199,827200
월ID,3,3,3,3,3
상점ID,25,25,25,25,25
상품ID,8092,7850,8051,8088,8089
월간 판매량,3,3,3,1,1
평균 판매가,149.0,199.0,30.0,199.0,199.0
판매건수,3,3,2,1,1
도시,13,13,13,13,13
상품분류ID,40,30,66,55,55
첫 판매월,0,0,0,0,0
대분류,5,3,8,7,7


필요 없는 피처 제거 후, 다운 캐스팅 진행

In [59]:
# 1.필요없는 피처 제거
# 첫 판매월, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

# 2. 데이터 다운 캐스팅
all_data = downcast(all_data, False) # 데이터 다운캐스팅

### 1-7. 피처 엔지니어링 7 : 마무리
- info()를 통해 최종적으로 all_data에 어떤 피처가 있는지 확인한다.

In [60]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8029765 entries, 827196 to 8856960
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float64
 14  평균 판매가_시차2               float64
 15  평균 판매가_시차3               float64
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상품I

- 다운 캐스팅과 가비지 컬렉션을 하지 않으면 여기서 메모리 사용량이 2GB가 넘는다.
- 모델 훈련 전부터 메모리 사용량을 초과해 코드가 멈추므로, 다운캐스팅과 가비지 컬렉션이 꼭 필요함.

In [61]:
# 베이스 라인과 마찬가지로, 데이터 나눠주고, 가비지 컬렉션 진행.

# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']

# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
del all_data
gc.collect();
 
# 베이스 라인에서는 타깃값에 clip() 했으나, 여기서는 앞서 미리 clipping 했으므로 적용하지 않는다.

### 2. 모델 훈련 및 성능 검증
- 피처 엔지니어링으로 30개 피처 얻었음!
- 이 데이터로 모델을 훈련하고 예측하여 결과를 체줄해본다.

#### 하이퍼 파라미터 조정
1. 조기 종료 조건 : 150번
2. 범주형 데이터 : 상점ID, 상품분류Id, + 도시, 대분류, 월 추가.

In [62]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse', 
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 10}

cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100) 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3657
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.0178	valid_1's rmse: 0.913728
[200]	training's rmse: 0.912629	valid_1's rmse: 0.841626
[300]	training's rmse: 0.859658	valid_1's rmse: 0.812341
[400]	training's rmse: 0.830685	valid_1's rmse: 0.800648
[500]	training's rmse: 0.811176	valid_1's rmse: 0.796129
[600]	training's rmse: 0.797613	valid_1's rmse: 0.794879
[700]	training's rmse: 0.78732	valid_1's rmse: 0.794375
[800]	training's rmse: 0.77898	valid_1's rmse: 0.794011
[900]	training's rmse: 0.771955	valid_1's rmse: 0.794313
[1000]	training's rmse: 0.765998	valid_1's rmse: 0.794295
Early stopping, best iteratio

case 1.
800번째 iteration까지 하고 조기종료됨.
670번째 iteration이 가장 우수했음. RMSE = 0.883922

case 2.
1000번째 iteration까지 하고 조기종료됨.
869번째 iteration이 가장 우수했음. RMSE = 0.79395

### 3. 예측 및 결과 제출
- 예측값 0~20 사이로 제한함.

In [63]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [64]:
# 메모리 절약을 위한 가비지 컬렉션
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();

case 1. Public score : 0.89534
case 2. Public score : 0.87701

- case2에서 훈련 데이터에서 테스트 데이터에 있는 상점ID만 추출하는 방법은 본 경진대회 같이 테스트 데이터가 고정돼 있는 경우에만 유효함.
- 테스트 데이터가 변할 수 있는 현업에서는 적합하지 않음. -> 특정 피처만 추출해서 훈련한 모델은 일반적인 상황에서 범용적으로 사용할 수 없는 모델이므로.